# ASTR 244W: Observation Notebook
The purpose of this notebook is to assist during observational time. This includes calculating how many observations to obtain a predetermiend S/N based on the current seeing, a built-in center-me, as well as any other tools used. 

In [1]:
import numpy as np
import matplotlib.pyplot
from astropy.table import QTable
import astropy.units as u
from astropy.constants import h, e, c


In [2]:
# Parameters of the Mees Telescope -- the Filters

# Background noise per pixel per exposure time
n_per_time = (140 * 1.35 / (600*u.s))


filt = np.array(["B", "Hbeta", "[OIII]", "G", "R", "Halpha", "[SII]"])
wavelength = np.array([442, 486.1, 500.7, 521, 633, 656.3, 670.0]) * u.nm
bandwidth = np.array([128, 8.5, 8.5, 80.5, 101, 7.0, 8.0]) * u.nm
flux_density = np.array([6.18e-8, 1.76e-8, 4.67e-8, 4.10e-8, 2.32e-8, 1.76e-8, 1.92e-8])*u.erg/u.s/u.cm/u.cm/u.nm
flux = np.array([(bandwidth[0]*flux_density[0]).value, 9.84e-8, 4.22e-7,
                 (bandwidth[3]*flux_density[3]).value, (bandwidth[4]*flux_density[4]).value, 
                 1.31e-7, 1.63e-7]) * u.erg / u.s / u.cm / u.cm
quantum_efficiency = np.array([0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6])
transmission_of_optical_system = np.array([0.95, 0.75, 0.75, 0.95, 0.95, 0.75, 0.75])

filters_information = QTable([filt, wavelength, bandwidth, flux_density, flux, quantum_efficiency, transmission_of_optical_system],
                  names = ('Filter', 'Wavelength', 'Bandwidth', 'Flux Density', 'Flux', 'Quantum Efficiency', 'Transmission of Optical System'),
                  meta= {'name': 'Filter parameters'})

#Filter binning in 2x2
pix_per_arc = (2 * 0.224 * u.arcsec )
collecting_area = 2700 * u.cm * u.cm

filters_information 

Filter,Wavelength,Bandwidth,Flux Density,Flux,Quantum Efficiency,Transmission of Optical System
,nm,nm,erg / (cm2 nm s),erg / (cm2 s),,
str6,float64,float64,float64,float64,float64,float64
B,442.0,128.0,6.18e-08,7.9104e-06,0.6,0.95
Hbeta,486.1,8.5,1.76e-08,9.84e-08,0.6,0.75
[OIII],500.7,8.5,4.67e-08,4.22e-07,0.6,0.75
G,521.0,80.5,4.1e-08,3.3005000000000004e-06,0.6,0.95
R,633.0,101.0,2.32e-08,2.3432e-06,0.6,0.95
Halpha,656.3,7.0,1.76e-08,1.31e-07,0.6,0.75
[SII],670.0,8.0,1.92e-08,1.63e-07,0.6,0.75


In [3]:
# Number of Pixels affected by seeing
def seeing_to_pixels(seeing):
    solid_angle = np.pi * seeing**2 / 4.0
    return np.ceil(solid_angle / pix_per_arc**2)

# Determine intensity of background light per electron charge
def I_B(N_pixels, t_exposure):
    result =  np.sqrt(N_pixels * (n_per_time * t_exposure)) / t_exposure
    return result

# Power needed to acheive a given SNR 
def P_SNR(desired_snr, seeing_pixel_num, t_exposure, tau, nu, lambda0):
    return (
        (desired_snr * h * c *  I_B(seeing_pixel_num, t_exposure) )
            / (tau * nu  * lambda0 )
            ).to(u.W)

def power_of_target(magnitude, filter):
    # Check if the filter is in the table
    if filter not in filters_information['Filter']:
        print(f'{filter} not in filters. Please choose from {filters_information["Filter"]}')
        return None
    loc = np.where(filters_information['Filter'] == filter)[0][0]


    F_mag = (filters_information[loc]['Flux'] * 10**(-magnitude/2.5)) * collecting_area
    return F_mag.to(u.W)

# Number of Observations of a target with a given magnitude in a given filter
def num_obs(magnitude, filter, desired_snr, seeing, t_exposure):
    seeing_pixel_num = seeing_to_pixels(seeing)
    power = power_of_target(magnitude, filter)
    if power is None:
        return None
    
    # Take lambda0 to be the wavelength of the filter
    lambda0 = filters_information[filters_information['Filter'] == filter]['Wavelength'][0]
    tau = filters_information[filters_information['Filter'] == filter]['Transmission of Optical System'][0]
    nu = filters_information[filters_information['Filter'] == filter]['Quantum Efficiency'][0]    

    P_desired = P_SNR(desired_snr, 
              seeing_pixel_num, 
              t_exposure,
              tau, 
              nu,
              lambda0)
    
    return P_desired / power 

def print_table(num_obs_tab, snr, filter, t_exposure):
    for i, v in enumerate(filter):
        print(f'For {v} filter: A total observation time of {num_obs_tab[:,i]} are needed for SNR of {snr}.')
    
# Determine the number of observations given various parameters
def num_obs_table(magnitude, seeing, t_exposure):
    snr = np.array([5, 10])
    filter = filters_information['Filter']
    num_obs_tab = np.zeros((len(snr), len(filters_information)))
    for i in range(len(snr)):
        for j in range(len(filter)):
            num_obs_tab[i][j] = num_obs(magnitude, filter[j], snr[i], seeing, t_exposure)
    num_obs_tab = num_obs_tab **2 * t_exposure
    print_table(num_obs_tab, snr, filter, t_exposure)

In [4]:
#Printing exposure information for a given filter for S/N 5 & 10
def filter_exp_info(exp_time, snr, Filter, t_exposure):
    for i, v in enumerate(Filter):
        time_5 = exp_time[0][i]
        time_10 = exp_time[1][i]
        print(f'{v} Filter')
        print(f'S/N = {snr[0]} : Total exposure time: {time_5}. Total exposure count: {np.ceil(time_5/t_exposure)}')
        print(f'S/N = {snr[1]}: Total exposure time: {time_10}. Total exposure count: {np.ceil(time_10/t_exposure)}')

#Calculating exposure information for all filters
def exposure_count(magnitude, seeing, t_exposure):
    snr = [5,10]
    Filter = filters_information['Filter']
    exposure_time = np.zeros((len(snr), len(Filter)))
    for i in range(len(snr)):
        for j in range(len(Filter)):
            exposure_time[i][j] = num_obs(magnitude, Filter[j], snr[i], seeing, t_exposure)
    
    exposure_time = exposure_time**2 * t_exposure
    
    print(f'Target Magnitude: {magnitude}')
    print(f'Seeing: {seeing}')
    print(f'Single Exposure Time: {t_exposure}')
    print('______________________________')
    filter_exp_info(exposure_time, snr, Filter, t_exposure)

In [12]:
#Example of exposure count
exposure_count(22, 3*u.arcsec, 300*u.s)

Target Magnitude: 22
Seeing: 3.0 arcsec
Single Exposure Time: 300.0 s
______________________________
B Filter
S/N = 5 : Total exposure time: 15.381116446867301 s. Total exposure count: 1.0
S/N = 10: Total exposure time: 61.524465787469204 s. Total exposure count: 1.0
Hbeta Filter
S/N = 5 : Total exposure time: 131859.8365504524 s. Total exposure count: 440.0
S/N = 10: Total exposure time: 527439.3462018096 s. Total exposure count: 1759.0
[OIII] Filter
S/N = 5 : Total exposure time: 6757.310687386839 s. Total exposure count: 23.0
S/N = 10: Total exposure time: 27029.242749547357 s. Total exposure count: 91.0
G Filter
S/N = 5 : Total exposure time: 63.590811549963384 s. Total exposure count: 1.0
S/N = 10: Total exposure time: 254.36324619985353 s. Total exposure count: 1.0
R Filter
S/N = 5 : Total exposure time: 85.46792316364551 s. Total exposure count: 1.0
S/N = 10: Total exposure time: 341.87169265458203 s. Total exposure count: 2.0
Halpha Filter
S/N = 5 : Total exposure time: 40813.7

###### Examples from class

In [6]:
# Examples completed in class
# 10th magnitude star in G filter
Example1 = power_of_target(10, 'G')
print(f'Power of 10th magnitude target in G filter: {Example1}')

# Number of Pixels effected by seeing of 2 arcsec
Example2 = seeing_to_pixels(2.0 * u.arcsec)
print(f'Number of pixels affected by seeing of 2 arcsec: {Example2}')

# Power needed to acheive S/N = 5
Example2Half = P_SNR(5, Example2, 600*u.s, 0.95, 0.45, 0.54 * u.um)

print(f'Power needed to acheive S/N = 5: {Example2Half}')

# Number of 600 second observations of 24th magnitude star in G filter to acheive S/N = 5
Example3 = num_obs(magnitude = 24,
                   filter = 'G',
                   desired_snr = 5,
                   seeing = 2.0 * u.arcsec,
                   t_exposure = 600*u.s)

print(f'f={Example3}')
print(f'Number of {600*u.s} exposures: {np.ceil(Example3**2)}')

print(f'Total observation time: {Example3**2 * 600 * u.s}')
print("--------------------------------------------")
print("--------------------------------------------\n")
print("For a 9.52 magnitude star:")
print("__________________________")
num_obs_table(magnitude = 20, seeing = 3.0 * u.arcsec, t_exposure = 300 * u.s)

Power of 10th magnitude target in G filter: 8.911350000000002e-14 W
Number of pixels affected by seeing of 2 arcsec: 16.0
Power needed to acheive S/N = 5: 3.943269874081179e-19 W
f=1.369399924253292
Number of 600.0 s exposures: 2.0
Total observation time: 1125.153691526953 s
--------------------------------------------
--------------------------------------------

For a 9.52 magnitude star:
__________________________
For B filter: A total observation time of [0.38635618 1.54542471] s are needed for SNR of [ 5 10].
For Hbeta filter: A total observation time of [ 3312.16934292 13248.67737169] s are needed for SNR of [ 5 10].
For [OIII] filter: A total observation time of [169.73597029 678.94388117] s are needed for SNR of [ 5 10].
For G filter: A total observation time of [1.59732897 6.38931587] s are needed for SNR of [ 5 10].
For R filter: A total observation time of [2.14685717 8.58742866] s are needed for SNR of [ 5 10].
For Halpha filter: A total observation time of [1025.19521917 4